In [ ]:
import googlemaps
import pandas as pd
import numpy as np
import gspread
from sklearn.neighbors import NearestNeighbors
#from matplotlib import pyplot as plt

In [ ]:
l = wb.worksheets()
wb.worksheets()

In [ ]:
x = get_worksheet_id(l, '2615 - North')
print(x)

In [ ]:
first_circuit.get_values(HEADER_RANGE)

In [ ]:
site_groups = ["1210 NE 77TH ST Seattle, WA",
"7804 12th Ave NE, Seattle, WA 98115",
"2306 NE 77TH ST Seattle, WA",
"7708 24TH AVE NE Seattle, WA"
"7722 24TH AVE NE Seattle, WA"
"90205 NE 91ST ST Seattle, WA"
"1740 NE 86TH ST Seattle, WA"
"1555 NE 90TH ST Seattle, WA",
"1719 NE 90TH ST Seattle, WA",
"1741 NE 90TH ST Seattle, WA",
"8927 20TH AVE NE Seattle, WA",
"8617 & 8623 20TH AVE NE Seattle, WA"]

In [ ]:
addresses = [['618 NW 47TH ST Seattle, WA'],
 ['612 NW 47TH ST Seattle, WA'],
 ['610 NW 47TH ST Seattle, WA'],
 ['643 NW 48TH ST Seattle, WA'],
 ['4914 8TH AVE NW Seattle, WA'],
 ['802 NW 49TH ST Seattle, WA'],
 ['5014 8TH AVE NW Seattle, WA'],
 ['651 NW 51ST ST Seattle, WA'],
 ['643 NW 51ST ST Seattle, WA'],
 ['603 NW 51ST ST Seattle, WA'],
 ['5104 8TH AVE NW Seattle, WA'],
 ['646 NW 52ND ST Seattle, WA'],
 ['633 NW 52ND ST Seattle, WA'],
 ['613 NW 52ND ST Seattle, WA'],
 ['605 NW 52ND ST Seattle, WA'],
 ['662 NW 52ND ST  Seattle, WA'],
 ['5218 8TH AVE NW Seattle, WA'],
 ['662 NW 53RD ST Seattle, WA'],
 ['654 NW 53RD ST  Seattle, WA'],
 ['636 NW 53RD ST Seattle, WA'],
 ['632 NW 53RD ST Seattle, WA'],
 ['616 NW 53RD ST Seattle, WA'],
 ['606 NW 53RD ST Seattle, WA'],
 ['602 NW 53RD ST Seattle, WA'],
 ['656 NW 54TH ST Seattle, WA'],
 ['1423 NW MARKET ST Seattle, WA'],
 ['1130 NW 54TH ST Seattle, WA'],
 ['644 NW 54TH ST Seattle, WA'],
 ['642 NW 54TH ST Seattle, WA'],
 ['640 NW 54TH ST Seattle, WA'],
 ['634 NW 54TH ST Seattle, WA'],
 ['614 NW 54TH ST Seattle, WA'],
 ['5403 6TH AVE NW Seattle, WA'],
 ['1140 NW 54TH ST Seattle, WA'],
 ['1138 NW 54TH ST Seattle, WA'],
 ['1129 NW MARKET ST Seattle, WA'],
 ['1139 NW MARKET ST Seattle, WA'],
 ['1126 NW MARKET ST Seattle, WA'],
 ['1130 NW MARKET ST Seattle, WA'],
 ['1115 NW MARKET ST Seattle, WA'],
 ['949 NW MARKET ST Seattle, WA'],
 ['942 NW MARKET ST Seattle, WA'],
 ['936 NW MARKET ST Seattle, WA'],
 ['922 NW MARKET ST Seattle, WA'],
 ['908 - 912 NW MARKET ST Seattle, WA'],
 ['845 NW MARKET ST Seattle, WA'],
 ['841 NW MARKET ST Seattle, WA'],
 ['844 NW MARKET ST Seattle, WA'],
 ['840 NW MARKET ST Seattle, WA'],
 ['836 NW MARKET ST Seattle, WA'],
 ['832 NW MARKET ST Seattle, WA'],
 ['828 NW MARKET ST Seattle, WA'],
 ['824 NW MARKET ST Seattle, WA'],
 ['820 NW MARKET ST Seattle, WA'],
 ['812 NW MARKET ST Seattle, WA'],
 ['808 NW MARKET ST Seattle, WA'],
 ['837 NW MARKET ST Seattle, WA'],
 ['833 NW MARKET ST Seattle, WA'],
 ['829 NW MARKET ST Seattle, WA'],
 ['821 NW MARKET ST  Seattle, WA'],
 ['717 NW 58TH ST Seattle, WA'],
 ['5604 8TH AVE NW  Seattle, WA'],
 ['5610 - 5608 8TH AVE NW Seattle, WA'],
 ['5612 - 5614 8TH AVE NW Seattle, WA'],
 ['5612 - 5618 8TH AVE NW Seattle, WA'],
 ['1400 NW 56TH ST Seattle, WA'],
 ['5614 14TH AVE NW Seattle, WA'],
 ['1151 NW 57TH ST Seattle, WA'],
 ['1133 NW 57TH ST Seattle, WA'],
 ['1101 NW 54TH DH Seattle, WA'],
 ['5609 11TH AVE NW Seattle, WA'],
 ['933 NW 57TH ST Seattle, WA'],
 ['929 NW 57TH ST Seattle, WA'],
 ['ACROSS FROM 901 NW 57TH ST Seattle, WA'],
 ['843 NW 57TH ST Seattle, WA'],
 ['811 NW 57TH ST Seattle, WA'],
 ['803 NW 57TH ST Seattle, WA'],
 ['5700 8TH AVE NW Seattle, WA'],
 ['5702 8TH AVE NW Seattle, WA'],
 ['5706 8TH AVE NW Seattle, WA'],
 ['5710 8th Ave NW, Seattle, WA 98107'],
 ['5712 8TH AVE NW Seattle, WA'],
 ['5718 8TH AVE NW Seattle, WA'],
 ['1415 NW 58TH ST Seattle, WA'],
 ['1409 NW 58TH ST Seattle, WA'],
 ['5555 14TH AVE NW Seattle, WA'],
 ['5712 14TH AVE NW Seattle, WA'],
 ['1133 NW 58TH ST  Seattle, WA'],
 ['1123 NW 58TH ST  Seattle, WA'],
 ['ACROSS FROM 943 NW 58TH ST Seattle, WA'],
 ['849 NW 58TH ST Seattle, WA'],
 ['5812 8TH AVE NW Seattle, WA'],
 ['5816 8TH AVE NW Seattle, WA'],
 ['811 NW 60TH ST Seattle, WA'],
 ['819 NW 60TH ST Seattle, WA'],
 ['5914 8TH AVE NW Seattle, WA'],
 ['5901 8th Ave NW, Seattle, WA 98107'],
 ['806 NW 61ST ST Seattle, WA'],
 ['6102 8TH AVE NW Seattle, WA'],
 ['6112 8TH AVE NW Seattle, WA'],
 ['6122 8TH AVE NW Seattle, WA'],
 ['802 NW 62ND ST Seattle, WA'],
 ['6130 8TH AVE NW Seattle, WA'],
 ['6400 8TH AVE NW Seattle, WA'],
 ['803 NW 64TH ST Seattle, WA'],
 ['802 NW 64TH ST Seattle, WA'],
 ['1439 NW 65TH ST Seattle, WA'],
 ['6407 14TH AVE NW Seattle, WA'],
 ['6418 14TH AVE NW Seattle, WA'],
 ['6512 12TH AVE NW Seattle, WA'],
 ['1203 NW 67TH ST Seattle, WA'],
 ['1112 NW 67TH ST Seattle, WA'],
 ['1108 NW 67TH ST Seattle, WA'],
 ['6722 12th Ave NW, Seattle, WA 98117'],
 ['6734 12TH AVE NW Seattle, WA'],
 ['6742 12TH AVE NW Seattle, WA'],
 ['6750 12TH AVE NW Seattle, WA 98117'],
 ['1101 NW 65TH ST Seattle, WA'],
 ['6513 11TH AVE NW Seattle, WA'],
 ['6527 11TH AVE NW Seattle, WA'],
 ['6530 11TH AVE NW Seattle, WA'],
 ['6545 11TH AVE NW Seattle, WA'],
 ['1019 NW 67TH ST Seattle, WA'],
 ['1015 NW 67TH ST Seattle, WA'],
 ['1007 NW 67TH ST  Seattle, WA'],
 ['6702 11TH AVE NW Seattle, WA'],
 ['6713 11TH AVE NW Seattle, WA'],
 ['6725 11TH AVE NW Seattle, WA'],
 ['6751 11TH AVE NW Seattle, WA'],
 ['6759 11TH AVE NW Seattle, WA'],
 ['6509 10TH AVE NW Seattle, WA'],
 ['867 NW 65TH ST Seattle, WA'],
 ['6514 DIBBLE AVE NW Seattle, WA'],
 ['6507 DIVISION AVE NW Seattle, WA'],
 ['826 NW 67TH ST Seattle, WA'],
 ['7016 DIBBLE AVE NW Seattle, WA'],
 ['820 NW 73RD ST Seattle, WA'],
 ['7318 DIBBLE AVE NW Seattle, WA'],
 ['ACROSS FROM 6516 DIVISION AVE NW Seattle, WA'],
 ['6520 DIVISION AVE NW Seattle, WA'],
 ['6541 CLEOPATRA PL NW Seattle, WA'],
 ['753 NW 67TH ST Seattle, WA'],
 ['752 - 750 NW 67TH ST Seattle, WA'],
 ['6707 CLEOPATRA PL NW Seattle, WA'],
 ['6717 CELOPATRA PL NW Seattle, WA'],
 ['6737 CLEOPATRA PL NW Seattle, WA'],
 ['6747 CLEOPATRA PL NW Seattle, WA'],
 ['7010 8TH AVE NW Seattle, WA'],
 ['7015 8TH AVE NW Seattle, WA'],
 ['7028 8TH AVE NW Seattle, WA'],
 ['ACROSS FROM 7040 8TH AVE NW Seattle, WA'],
 ['7056 8TH AVE NW Seattle, WA'],
 ['753 NW 73RD ST Seattle, WA'],
 ['7318 8TH AVE NW Seattle, WA'],
 ['641 NW 74TH ST Seattle, WA'],
 ['631 NW 74TH ST Seattle, WA'],
 ['623 NW 74TH ST Seattle, WA'],
 ['619 NW 74TH ST Seattle, WA'],
 ['607 NW 74TH ST Seattle, WA'],
 ['7317 6TH AVE NW Seattle, WA'],
 ['656 NW 74TH ST Seattle, WA'],
 ['7412 8TH AVE NW Seattle, WA'],
 ['7506 8TH AVE NW Seattle, WA'],
 ['655 NW 76TH ST Seattle, WA'],
 ['643 NW 76TH ST Seattle, WA'],
 ['637 NW 76TH ST Seattle, WA'],
 ['7612 8TH AVE NW Seattle, WA'],
 ['7618 8TH AVE NW Seattle, WA'],
 ['651 NW 77TH ST Seattle, WA'],
 ['643 NW 77TH ST Seattle, WA 98117'],
 ['621 NW 77TH ST Seattle, WA'],
 ['619 NW 77TH ST Seattle, WA'],
 ['7714 8TH AVE NW Seattle, WA'],
 ['7631 8TH AVE NW Seattle, WA'],
 ['7812 8TH AVE NW Seattle, WA'],
 ['7816 8TH AVE NW Seattle, WA'],
 ['324 NW 80TH ST Seattle, WA'],
 ['328 NW 80TH ST Seattle, WA'],
 ['330 NW 80TH ST Seattle, WA'],
 ['346 NW 80TH ST Seattle, WA'],
 ['356 NW 80TH ST Seattle, WA'],
 ['606 NW 80TH ST Seattle, WA'],
 ['616 NW 80TH ST Seattle, WA'],
 ['628 NW 80TH ST Seattle, WA'],
 ['630 NW 80TH ST Seattle, WA'],
 ['640 NW 80TH ST  Seattle, WA'],
 ['7046 CLEPOATRA PL NW Seattle, WA'],
 ['7038 CLEOPATRA PL NW Seattle, WA'],
 ['6750 CLEOPATRA PL NW Seattle, WA'],
 ['6742 CLEPOATRA PL NW Seattle, WA'],
 ['6734 CLEPPATRA PL NW Seattle, WA'],
 ['6730 CLEOPATRA PL NW Seattle, WA'],
 ['6722 CLEOPATRA PL NW Seattle, WA'],
 ['6714 CLEOPATRA PL NW Seattle, WA'],
 ['6712 CLEOPATRA PL NW Seattle, WA'],
 ['6528 CLEOPATRA PL NW Seattle, WA'],
 ['6518 CLEOPATRA PL NW Seattle, WA'],
 ['713 NW 73RD ST Seattle, WA'],
 ['864 NW 70TH ST Seattle, WA'],
 ['7008 9TH AVE NW Seattle, WA'],
 ['7318 9TH AVE NW Seattle, WA'],
 ['7322 9TH AVE NW Seattle, WA'],
 ['7326 9TH AVE NW Seattle, WA'],
 ['904 NW 73RD ST  Seattle, WA'],
 ['7008 10TH AVE NW Seattle, WA'],
 ['7012 10TH AVE NW Seattle, WA'],
 ['7310 10TH AVE NW Seattle, WA'],
 ['7302 11TH AVE NW Seattle, WA'],
 ['7058 11TH AVE NW Seattle, WA'],
 ['1102 NW 73RD ST  Seattle, WA'],
 ['7302 12TH AVE NW Seattle, WA'],
 ['7006 12TH AVE NW Seattle, WA'],
 ['7016 12TH AVE NW Seattle, WA'],
 ['7022 12TH AVE NW Seattle, WA'],
 ['7026 12TH AVE NW Seattle, WA'],
 ['7032 12TH AVE NW Seattle, WA'],
 ['7036 12TH AVE NW Seattle, WA'],
 ['7049 12TH AVE NW Seattle, WA'],
 ['7354 12TH AVE NW Seattle, WA'],
 ['1216 NW 73RD ST Seattle, WA'],
 ['1216 NW 70TH ST Seattle, WA'],
 ['6758 13TH AVE NW Seattle, WA'],
 ['7054 13TH AVE NW Seattle, WA'],
 ['7058 13TH AVE NW Seattle, WA'],
 ['7302 14TH AVE NW Seattle, WA'],
 ['6718 13TH AVE NW Seattle, WA'],
 ['6728 13TH AVE NW Seattle, WA'],
 ['6738 13TH AVE NW Seattle, WA'],
 ['6741 13TH AVE NW Seattle, WA'],
 ['6748 13TH AVE NW Seattle, WA'],
 ['6740 - 6742 14TH AVE NW Seattle, WA'],
 ['7002 ALONZO AVE NW Seattle, WA'],
 ['7020 ALONZO AVE NW Seattle, WA'],
 ['7024 ALONZO AVE NW Seattle, WA'],
 ['7037 ALONZO AVE NW Seattle, WA'],
 ['7052 ALONZO AVE NW Seattle, WA'],
 ['7318 ALONZO AVE NW Seattle, WA'],
 ['7342 ALONZO AVE NW Seattle, WA'],
 ['7059 ALONZO AVE NW Seattle, WA'],
 ['6719 ALONZO AVE NW Seattle, WA'],
 ['7008 MARY AVE NW Seattle, WA'],
 ['7016 MARY AVE NW Seattle, WA'],
 ['7022 MARY AVE NW Seattle, WA'],
 ['7054 MARY AVE NW Seattle, WA'],
 ['7060 MARY AVE NW Seattle, WA'],
 ['1450 NW 73RD ST Seattle, WA'],
 ['1470 NW 67TH ST Seattle, WA'],
 ['6712 MARY AVE NW Seattle, WA'],
 ['6748 MARYAVE NW Seattle, WA'],
 ['7549 15TH AVE NW Seattle, WA'],
 ['7545 15TH AVE NW Seattle, WA'],
 ['7543 15TH AVE NW Seattle, WA'],
 ['7525 15TH AVE NW Seattle, WA'],
 ['7515 15TH AVE NW Seattle, WA'],
 ['7509 15TH AVE NW Seattle, WA'],
 ['7341 15TH AVE NW Seattle, WA'],
 ['7348 15TH AVE NW Seattle, WA'],
 ['7342 15TH AVE NW Seattle, WA'],
 ['7315 15TH AVE NW Seattle, WA'],
 ['7314 15TH AVE NW Seattle, WA'],
 ['7052 15TH AVE NW Seattle, WA'],
 ['7048 15TH AVE NW Seattle, WA'],
 ['7043 15TH AVE NW Seattle, WA'],
 ['7039 15TH AVE NW Seattle, WA'],
 ['7033 15TH AVE NW Seattle, WA'],
 ['7023 15TH AVE NW Seattle, WA'],
 ['6743 15TH AVE NW Seattle, WA'],
 ['6719 15TH AVE NW Seattle, WA'],
 ['6722 15TH AVE NW Seattle, WA'],
 ['6701 15TH AVE NW Seattle, WA'],
 ['6706 15TH AVE NW Seattle, WA'],
 ['6559 15TH AVE NW Seattle, WA'],
 ['6525 15TH AVE NW Seattle, WA'],
 ['6515 - 6523 15TH AVE NW Seattle, WA'],
 ['7726 16TH AVE NW Seattle, WA'],
 ['7720 16TH AVE NW Seattle, WA'],
 ['7718 16TH AVE NW Seattle, WA'],
 ['7706 16TH AVE NW Seattle, WA'],
 ['7704 16TH AVE NW Seattle, WA'],
 ['7702 16TH AVE NW Seattle, WA'],
 ['7753 15TH AVE NW Seattle, WA'],
 ['7750 16TH AVE NW Seattle, WA 98117'],
 ['7546 16TH AVE NW Seattle, WA'],
 ['7522 16TH AVE NW Seattle, WA'],
 ['7510 16TH AVE NW Seattle, WA'],
 ['7360 17TH AVE NW Seattle, WA'],
 ['7502 17TH AVE NW Seattle, WA'],
 ['7361 16TH AVE NW Seattle, WA'],
 ['7360 16TH AVE NW Seattle, WA'],
 ['7354 16TH AVE NW Seattle, WA'],
 ['7030 16TH AVE NW Seattle, WA'],
 ['7022 16TH AVE NW Seattle, WA'],
 ['7010 16TH AVE NW Seattle, WA'],
 ['ACROSS FROM 6756 18TH AVE NW Seattle, WA'],
 ['6759 16TH AVE NW Seattle, WA'],
 ['6756 16TH AVE NW Seattle, WA'],
 ['6746 16TH AVE NW Seattle, WA'],
 ['6740 16TH AVE NW Seattle, WA'],
 ['6732 16TH AVE NW Seattle, WA'],
 ['7306 16TH AVE NW Seattle, WA'],
 ['1612 NW 73RD ST Seattle, WA'],
 ['7302 17TH AVE NW Seattle, WA'],
 ['1702 NW 73RD ST Seattle, WA'],
 ['7306 18TH AVE NW Seattle, WA'],
 ['7305 18TH AVE NW Seattle, WA'],
 ['7751 18TH AVE NW Seattle, WA'],
 ['7725 18TH AVE NW Seattle, WA'],
 ['7701 18TH AVE NW Seattle, WA'],
 ['1814 NW 77TH ST Seattle, WA'],
 ['7559 18TH AVE NW Seattle, WA'],
 ['7541 18TH AVE NW Seattle, WA'],
 ['7537 18TH AVE NW Seattle, WA'],
 ['7521 18TH AVE NW Seattle, WA'],
 ['7515 18TH AVE NW Seattle, WA'],
 ['W7351 18TH AVW NW Seattle, WA'],
 ['7335 18TH AVE NW Seattle, WA'],
 ['7329 18TH AVE NW Seattle, WA'],
 ['7319 18TH AVE NW Seattle, WA'],
 ['7302 19TH AVE NW Seattle, WA'],
 ['7703 19TH AVE NW Seattle, WA'],
 ['7725 19TH AVE NW Seattle, WA'],
 ['7361 19TH AVE NW Seattle, WA'],
 ['7360 19TH AVE NW Seattle, WA'],
 ['7353 19TH AVE NW Seattle, WA'],
 ['7347 19TH AVE NW Seattle, WA'],
 ['7345 19TH AVE NW Seattle, WA'],
 ['7301 19TH AVE NW Seattle, WA'],
 ['7318 20TH AVE NW Seattle, WA 98117'],
 ['7324 20TH AVE NW Seattle, WA'],
 ['7338 20TH AVE NW Seattle, WA'],
 ['7342 20TH AVE NW Seattle, WA'],
 ['7360 21ST AVE NW Seattle, WA'],
 ['7310 20TH AVE NW Seattle, WA'],
 ['7721 19TH AVE NW Seattle, WA'],
 ['7727 19TH AVE NW Seattle, WA'],
 ['7739 19TH AVE NW Seattle, WA'],
 ['7747 19TH AVE NW Seattle, WA'],
 ['7702 20TH AVE NW Seattle, WA'],
 ['7712 20TH AVE NW Seattle, WA'],
 ['7726 20TH AVE NW Seattle, WA'],
 ['7730 20TH AVE NW Seattle, WA'],
 ['7732 20TH AVE NW Seattle, WA'],
 ['7746 20TH AVE NW Seattle, WA'],
 ['7752 20TH AVE NW Seattle, WA'],
 ['7701 20TH AVE NW Seattle, WA'],
 ['7705 20TH AVE NW Seattle, WA'],
 ['7721 20TH AVE NW Seattle, WA'],
 ['7724 21TS AVE NW Seattle, WA'],
 ['7728 21ST AVE NW Seattle, WA'],
 ['2004 NW 73RD ST Seattle, WA'],
 ['2016 NW 73RD ST Seattle, WA'],
 ['7349 21ST AVE NW Seattle, WA'],
 ['7345 21ST AVE NW Seattle, WA'],
 ['7329 21ST AVE NW Seattle, WA'],
 ['7321 21ST AVE NW Seattle, WA'],
 ['7325 21ST AVE NW Seattle, WA'],
 ['7311 21ST AVE NW Seattle, WA'],
 ['7303 21ST AVE NW Seattle, WA'],
 ['7047 21ST AVE NW Seattle, WA'],
 ['7302 22ND AVE NW Seattle, WA'],
 ['7032 22ND AVE NW Seattle, WA'],
 ['7048 22ND AVE NW Seattle, WA'],
 ['7312 22ND AVE NW Seattle, WA'],
 ['7330 22ND AVE NW Seattle, WA'],
 ['7340 22ND AVE NW Seattle, WA'],
 ['6704 23RD AVE NW Seattle, WA'],
 ['6710 23RD AVE NW Seattle, WA'],
 ['6716 23RD AVE NW Seattle, WA'],
 ['6722 23RD AVE NW Seattle, WA'],
 ['6740 23RD AVE NW Seattle, WA'],
 ['2353 NW 70TH ST Seattle, WA'],
 ['2317 NW 70TH ST Seattle, WA'],
 ['2315 NW 70TH ST Seattle, WA'],
 ['6756 23RD AVE NW Seattle, WA'],
 ['2207 NW 70TH ST Seattle, WA'],
 ['7016 23RD AVE NW Seattle, WA'],
 ['7302 23RD AVE NW Seattle, WA'],
 ['7318 23RD AVE NW Seattle, WA'],
 ['7332 23RD AVE NW Seattle, WA'],
 ['7341 23RD AVE NW Seattle, WA'],
 ['7359 23RD AVE NW Seattle, WA'],
 ['7302 JONES AVE NW Seattle, WA'],
 ['7358 JONES AVE NW Seattle, WA'],
 ['7503 JONES AVE NW Seattle, WA'],
 ['7343 JONES AVE NW Seattle, WA'],
 ['7309 JONES AVE NW Seattle, WA'],
 ['7301 JONES AVE NW Seattle, WA'],
 ['7053 JONES AVE NW Seattle, WA'],
 ['7049 JONES AVE NW Seattle, WA'],
 ['7035 JONES AVE NW Seattle, WA'],
 ['7033 JONES AVE NW Seattle, WA'],
 ['7015 JONES AVE NW Seattle, WA'],
 ['2360 NW 73RD ST Seattle, WA'],
 ['2416 NW 67TH ST Seattle, WA'],
 ['6736 25TH AVE NW Seattle, WA'],
 ['6740 25TH AVE NW Seattle, WA'],
 ['6750 25TH AVE NW Seattle, WA'],
 ['7002 25TH AVE NW Seattle, WA'],
 ['7003 25TH AVE NW Seattle, WA'],
 ['7014 25TH AVE NW Seattle, WA'],
 ['7024 25TH AVE NW Seattle, WA'],
 ['7036 25TH AVE NW Seattle, WA'],
 ['7305 25TH AVE NW Seattle, WA'],
 ['7322 25TH AVE NW Seattle, WA'],
 ['7302 26TH AVE NW Seattle, WA'],
 ['7349-7351 26TH AVE NW Seattle, WA'],
 ['7348 26TH AVE NW Seattle, WA'],
 ['7338 26TH AVE NW Seattle, WA'],
 ['7339 26TH AVE NW Seattle, WA'],
 ['7327 26TH AVE NW Seattle, WA'],
 ['7323 26TH AVE NW Seattle, WA'],
 ['7317 26TH AVE NW Seattle, WA'],
 ['7311 26TH AVE NW Seattle, WA'],
 ['7056 26TH AVE NW Seattle, WA'],
 ['7052 26TH AVE NW Seattle, WA'],
 ['7048 26TH AVE NW Seattle, WA'],
 ['7026 26TH AVE NW Seattle, WA'],
 ['7022 26TH AVE NW Seattle, WA'],
 ['7018 26TH AVE NW Seattle, WA'],
 ['7012 26TH AVE NW Seattle, WA'],
 ['7000 26TH AVE NW Seattle, WA'],
 ['6758 26TH AVE NW Seattle, WA'],
 ['6732 26TH AVE NW Seattle, WA'],
 ['6722 26TH AVE NW Seattle, WA'],
 ['6708 26TH AVE NW Seattle, WA'],
 ['7003 26TH AVE NW Seattle, WA'],
 ['2616 NW 70TH ST Seattle, WA'],
 ['7004 EARL AVE NW Seattle, WA'],
 ['7003 EARL AVE NW Seattle, WA'],
 ['6734 27TH AVE NW Seattle, WA'],
 ['6740 27TH AVE NW Seattle, WA'],
 ['6746 27TH AVE NW Seattle, WA'],
 ['6754 27TH AVE NW Seattle, WA'],
 ['7006 27TH AVE NW Seattle, WA'],
 ['7344 27TH AVE NW Seattle, WA'],
 ['7352 27TH AVE NW Seattle, WA'],
 ['7301 27TH AVE NW Seattle, WA'],
 ['7345 EARL AVE NW Seattle, WA'],
 ['7337 EARL AVE NW Seattle, WA'],
 ['7321 EARL AVE NW Seattle, WA'],
 ['7009 EARL AVE NW Seattle, WA'],
 ['6759 EARL AVE NW Seattle, WA'],
 ['6741 EARL AVE NW Seattle, WA'],
 ['6733 EARL AVE NW Seattle, WA'],
 ['6729 EARL AVE NW Seattle, WA 98117'],
 ['6725 EARL AVE NW Seattle, WA'],
 ['6814 28TH AVE NW Seattle, WA'],
 ['6908 8TH AVE NW Seattle, WA 98117'],
 ['2800 NW 70TH ST Seattle, WA'],
 ['2808 NW 70TH ST Seattle, WA'],
 ['7100 28TH AVE NW Seattle, WA'],
 ['7112 28TH AVE NW Seattle, WA'],
 ['7120 28TH AVE NW Seattle, WA'],
 ['7202 28TH AVE NW Seattle, WA'],
 ['7302 28TH AVE NW Seattle, WA'],
 ['7418 28TH AVE NW Seattle, WA'],
 ['7514 28TH AVE NW Seattle, WA'],
 ['7526 28TH AVE NW Seattle, WA'],
 ['2915 NW 75TH ST Seattle, WA'],
 ['7506 34TH AVE NW Seattle, WA'],
 ['7324 34TH AVE NW Seattle, WA'],
 ['7323 33RD AVE NW Seattle, WA'],
 ['7541 33RD AVE NW Seattle, WA'],
 ['7547 33RD AVE NW Seattle, WA'],
 ['7555 33RD AVE NW Seattle, WA'],
 ['3300 NW 77TH ST Seattle, WA'],
 ['8325 31ST AVE NW Seattle, WA'],
 ['3100 NW 80TH ST Seattle, WA'],
 ['7749 31ST AVE NW Seattle, WA'],
 ['7735 31ST AVE NW Seattle, WA'],
 ['7725 31ST AVE NW Seattle, WA'],
 ['7717 31ST AVE NW Seattle, WA'],
 ['7711 31ST AVE NW Seattle, WA'],
 ['7543 31ST AVE NW Seattle, WA'],
 ['7511 31ST AVE NW Seattle, WA'],
 ['3102 NW 75TH ST Seattle, WA'],
 ['7403 30TH AVE NW Seattle, WA'],
 ['2822 NW 74TH ST Seattle, WA'],
 ['2842 NW 74TH ST Seattle, WA'],
 ['3030 NW 74TH ST Seattle, WA'],
 ['3048 NW 74TH ST Seattle, WA'],
 ['7400 32ND AVE NW Seattle, WA'],
 ['3210 NW 74TH ST Seattle, WA'],
 ['3211 NW 73RD ST Seattle, WA'],
 ['7215 32ND AVE NW Seattle, WA'],
 ['3033 NW 73RD ST Seattle, WA'],
 ['7215 30TH AVE NW Seattle, WA'],
 ['2823 NW 73RD ST Seattle, WA'],
 ['2815 NW 73RD ST Seattle, WA'],
 ['7217 28TH AVE NW Seattle, WA'],
 ['7205 28TH AVE NW Seattle, WA'],
 ['2826 NW 72ND ST Seattle, WA'],
 ['2846 NW 72ND ST Seattle, WA'],
 ['2852 NW 72ND ST Seattle, WA']]

In [ ]:
import random
#548
lats = []
lngs = []
for i in range(548):
    n = random.randint(0,100)
    m = random.randint(0,100)
    lats.append(n)
    lngs.append(m)

In [ ]:
stacked = stack_coords(addresses, lats, lngs)
len(a)

In [ ]:
distance_matrix = precompute_distance_matrix(stacked, street_change_penalty)
knn = NearestNeighbors(n_neighbors=15, metric='precomputed')
knn.fit(distance_matrix)
_, neighbors_mat = knn.kneighbors(distance_matrix)

In [ ]:
group1 = []
for idx, x in enumerate(addresses):
    if idx in neighbors_mat[99]:
        group1.append(x)

In [ ]:
l = [1,2,3,4,5,6,7,8]
l[-4]

In [ ]:
l = [1,2,3,4,5,6,7,8]
l[-4]

In [ ]:
group1

In [ ]:
street_change_penalty = 100


def stack_coords(addresses, lats, lngs):
    split = [x[0].split() for x in addresses]
    streets = [x[1:4] for x in split]
    streets = ["".join(x).lower() for x in streets]
    stacked = [(street, lat, lng) for street, lat, lng in zip(streets, lats, lngs)]
    return stacked

def get_neighbors(points):
    coords = [(lat, lng) for street, lat, lng in points]
    
    knn = NearestNeighbors(
        n_neighbors=15,
        metric=lambda a, b: custom_distance(
        points[a], points[b], street_change_penalty),
    )

    # Fit kNN with just coordinates
    knn.fit(coords)

    # When calculating distances, use the full points including street information
    distance_mat, neighbors_mat = knn.kneighbors(coords)
    neighbors_mat = list(
        reversed(neighbors_mat[0])
    )  # reversed so closest pt comes last
    return neighbors_mat

def precompute_distance_matrix(points, penalty):
    num_points = len(points)
    distance_matrix = np.zeros((num_points, num_points))

    for i in range(num_points):
        for j in range(num_points):
            if i != j:
                street_i = points[i][0]
                street_j = points[j][0]
                coords_i = points[i][1:]
                coords_j = points[j][1:]
                distance = np.linalg.norm(np.array(coords_i) - np.array(coords_j))
                if street_i != street_j:
                    distance += penalty
                distance_matrix[i, j] = distance
    
    return distance_matrix

In [ ]:
orig_data

In [ ]:
    df = data.loc[:,'full_address']
    df = pd.DataFrame(df)
    addresses = df.values.tolist()

In [ ]:
#"site_no", 'address','work_description', 'owner_phone_comments', 'no_parks', 'nbw', 'projected_hours','flagging', 'requires_squirt_boom','merge', 'notes', 'also_clear_for'
fake_neighbors_mat = [1,4,9,7,500,6,78,333,245,123,45, 400, 380]

one_site_group = []
for idx in range(len(addresses)):
    if idx in fake_neighbors_mat:
        to_add = orig_data.iloc[idx].tolist()
        if to_add[0]:
            to_add[0] = "Site Number: " + to_add[0]
        if to_add[1]:
            to_add[1] = "Address: " + to_add[1]
        if to_add[2]:
            to_add[2] = "Work Description: " + to_add[2]
        if to_add[3]:
            to_add[3] = "Owner/Phone/Comments: " + to_add[3]
            
        if to_add[4]:
            to_add[4] = "No Parks: Yes"
        if not to_add[4]:
            to_add[4] = ""
            
        if to_add[5]:
            to_add[5] = "NBW: Yes"
        if not to_add[5]:
            to_add[5] = ""
        
        if to_add[6]:
            to_add[6] = "Projected Hours: " + to_add[6]
            
        if to_add[7]:
            to_add[7] = "Flagging: Yes"
        if not to_add[7]:
            to_add[7] = ""
        
        if to_add[8]:
            to_add[8] = "Squirt Boom: Yes"
        if not to_add[8]:
            to_add[8] = ""
        
        if to_add[9]:
            to_add[9] = "Merge: Yes"
        if not to_add[9]:
            to_add[9] = ""
        
        if to_add[10]:
            to_add[10] = "Notes: " + to_add[10]
        if to_add[11]:
            to_add[11] = "Also Clear For: " + to_add[11]
            
        to_add[12] = ''
        
        to_add = [x for x in to_add if x != '']
        one_site_group.append(to_add)

one_site_group



In [ ]:
ad_test = [ele for idx, ele in enumerate(addresses) if idx in fake_neighbors_mat]
ad_test

In [ ]:
def get_key(val):
    for key, value in neighbors_time_dict.items():
            if val == value:
                yield key

In [ ]:
clean_site_groups = []

for i in site_groups:
    groups = [
        x[0] for x in i
    ]
    clean_site_groups.append(groups)

In [ ]:
x = np.array(group_lngs[0])
y = np.array(group_lats[0])
plt.scatter(x,y)


x = np.array(group_lngs[1])
y = np.array(group_lats[1])
plt.scatter(x,y)


x = np.array(group_lngs[2])
y = np.array(group_lats[2])
plt.scatter(x,y)


x = np.array(group_lngs[3])
y = np.array(group_lats[3])
plt.scatter(x,y)


x = np.array(group_lngs[4])
y = np.array(group_lats[4])
plt.scatter(x,y)


x = np.array(group_lngs[5])
y = np.array(group_lats[5])
plt.scatter(x,y)


x = np.array(group_lngs[6])
y = np.array(group_lats[6])
plt.scatter(x,y)


x = np.array(group_lngs[7])
y = np.array(group_lats[7])
plt.scatter(x,y)


x = np.array(group_lngs[8])
y = np.array(group_lats[8])
plt.scatter(x,y)


x = np.array(group_lngs[9])
y = np.array(group_lats[9])
plt.scatter(x,y)


x = np.array(group_lngs[10])
y = np.array(group_lats[10])
plt.scatter(x,y)


x = np.array(group_lngs[11])
y = np.array(group_lats[11])
plt.scatter(x,y)


x = np.array(group_lngs[12])
y = np.array(group_lats[12])
plt.scatter(x,y)


x = np.array(group_lngs[13])
y = np.array(group_lats[13])
plt.scatter(x,y)


x = np.array(group_lngs[14])
y = np.array(group_lats[14])
plt.scatter(x,y)


x = np.array(group_lngs[15])
y = np.array(group_lats[15])
plt.scatter(x,y)


x = np.array(group_lngs[16])
y = np.array(group_lats[16])
plt.scatter(x,y)


x = np.array(group_lngs[17])
y = np.array(group_lats[17])
plt.scatter(x,y)


x = np.array(group_lngs[18])
y = np.array(group_lats[18])
plt.scatter(x,y)


x = np.array(group_lngs[19])
y = np.array(group_lats[19])
plt.scatter(x,y)


x = np.array(group_lngs[20])
y = np.array(group_lats[20])
plt.scatter(x,y)


x = np.array(group_lngs[21])
y = np.array(group_lats[21])
plt.scatter(x,y)


x = np.array(group_lngs[22])
y = np.array(group_lats[22])
plt.scatter(x,y)


x = np.array(group_lngs[23])
y = np.array(group_lats[23])
plt.scatter(x,y)

In [ ]:
for x in range(22):
    print(f"""
    x = np.array(group_lngs[{x}])
    y = np.array(group_lats[{x}])
    plt.scatter(x,y)
    """)